In [ ]:
!pip install pyarrow==14


In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# Langchain components tp use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support For Dataset retrieval with hugging face
from datasets import load_dataset

#With Cassio, the engine powering the Astra DB integration in Langchain,
# you will also initialize the DB connection:
import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [47]:
from google.colab import userdata
ASTRA_DB_APPLICATION_TOKEN = userdata.get('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_ID = userdata.get('ASTRA_DB_ID')

OPENAI_API_KEY =userdata.get('OPENAI_API_KEY')


In [48]:
# provide the path of pdf file

pdfreader = PdfReader('/content/spice.pdf')

In [49]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [50]:
raw_text

'Spices\nSpices at a central market in Agadir,\nMorocco\nA group of Indian herbs and spices in\nbowls\nSpices of Saúde flea market, São\nPaulo, BrazilSpice\nA spice is a seed, fruit, root, bark, or other plant substance\nprimarily used for flavoring or coloring food. Spices are\ndistinguished from herbs, which are the leaves, flowers, or stems of\nplants used for flavoring or as a garnish. Spices are sometimes used\nin medicine, religious rituals, cosmetics, or perfume produc tion.\nFor example, vanilla is commonly used as an ingredient in\nfragrance manufacturing.[1]\nA spice may be available in several forms: fresh, whole-dried, or\npre-ground dried. Generally, spices are dried. Spices may be\nground into a powder for conve nience. A whole dried spice has the\nlonge st shelf life, so it can be purchased and stored in larger\namounts, making it cheaper on a per-serving basis. A fresh spice,\nsuch as ginger, is usually more flavorful than its dried form, but\nfresh spices are more expe

In [51]:
# Intializing the connection to database

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id = ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137715314746528) 8795dbbf-3a5b-47fd-869e-67b6adafd259-us-east1.db.astra.datastax.com:29042:bcad06fc-5971-46aa-be36-c8f73c3f8329> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [37]:
# Creating Langchain embedding and LLM objects for later usage:

llm = OpenAI(openai_api_key= OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key= OPENAI_API_KEY)

In [39]:
# Creating  Langchain vector store ... backend by Astra DB

astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [42]:
from langchain.text_splitter import CharacterTextSplitter

# splitting the text using textspilitter such that it should increase the token size

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap =200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [43]:
texts[:50]

['Spices\nSpices at a central market in Agadir,\nMorocco\nA group of Indian herbs and spices in\nbowls\nSpices of Saúde flea market, São\nPaulo, BrazilSpice\nA spice is a seed, fruit, root, bark, or other plant substance\nprimarily used for flavoring or coloring food. Spices are\ndistinguished from herbs, which are the leaves, flowers, or stems of\nplants used for flavoring or as a garnish. Spices are sometimes used\nin medicine, religious rituals, cosmetics, or perfume produc tion.\nFor example, vanilla is commonly used as an ingredient in\nfragrance manufacturing.[1]\nA spice may be available in several forms: fresh, whole-dried, or\npre-ground dried. Generally, spices are dried. Spices may be\nground into a powder for conve nience. A whole dried spice has the',
 'A spice may be available in several forms: fresh, whole-dried, or\npre-ground dried. Generally, spices are dried. Spices may be\nground into a powder for conve nience. A whole dried spice has the\nlonge st shelf life, so it

In [44]:
# loading the top 50 words to the vector store

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 22 headlines.


In [45]:
first_question = True
while True:
  if first_question:
    query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
  else:
    query_text = input("\nWhat's your next your question (or type 'quit' to exit): ").strip()

  if query_text.lower() == "quit":
    break

  if query_text == "":
    continue

  first_question = False

  print("\nQUESTION: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \%s\"\n" % answer)


  print("FIRST DOCUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what is mortar?

QUESTION: "what is mortar?"


ANSWER: \Mortar is a tool used for grinding spices and other ingredients, consisting of a bowl and a pestle."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8725] "Pumpkin pie spice (United States)
Quatre épices (France)
Ras el hanout (North Africa ..."
    [0.8725] "Pumpkin pie spice (United States)
Quatre épices (France)
Ras el hanout (North Africa ..."
    [0.8633] "in its own hand gr inder or mill.
The flavor of a spice is derived in part from
comp ..."
    [0.8633] "in its own hand gr inder or mill.
The flavor of a spice is derived in part from
comp ..."

What's your next your question (or type 'quit' to exit): quit
